In [ ]:
import pandas

adult = pandas.read_table('data/adult.data', skipinitialspace=True, sep=',')

adult.shape

(32561, 15)

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2

train, test = train_test_split(adult, test_size=test_size, shuffle=False)

train.shape[0], test.shape[0]

(26048, 6513)

In [ ]:
from util import add_extra_rows

tester = add_extra_rows(test)

tester.shape[0]

6517

In [ ]:
from tensorflow_data_validation import generate_statistics_from_dataframe

trainStats = generate_statistics_from_dataframe(train)

testerStats = generate_statistics_from_dataframe(tester)

testStats = generate_statistics_from_dataframe(test)

In [ ]:
statlist = {}

statlist['rhs_statistics'] = trainStats

statlist['lhs_statistics'] = testStats

```python
from tensorflow_data_validation import visualize_statistics

visualize_statistics(trainStats)
```

```python
from tensorflow_data_validation import visualize_statistics

statlist['rhs_name'] = 'trainStats'

statlist['lhs_name'] = 'testStats'

visualize_statistics(**statlist)
```

```python
statlist['lhs_statistics'] = testerStats

statlist['lhs_name'] = 'testerStats'

visualize_statistics(**statlist)
```

In [ ]:
from tensorflow_data_validation import infer_schema, validate_statistics

trainSchema = infer_schema(trainStats)

```python

from tensorflow_data_validation import display_anomalies

display_anomalies(testAnomalies)

display_anomalies(testerAnomalies)
```

In [ ]:
from tensorflow_data_validation import display_anomalies

testAnomalies = validate_statistics(testStats, trainSchema)

type(testAnomalies.anomaly_info)

google.protobuf.pyext._message.MessageMapContainer

In [ ]:
False if len(testAnomalies.anomaly_info.items()) == 0 else True

False

In [ ]:
from tensorflow_data_validation import display_anomalies

testerAnomalies = validate_statistics(testerStats, trainSchema)

False if len(testerAnomalies.anomaly_info.items()) == 0 else True

True

In [ ]:
display_anomalies(testerAnomalies)

In [ ]:
from tensorflow_data_validation import get_domain

raceValues = get_domain(trainSchema, 'race').value

assert 'Asian' not in raceValues, 'False'

In [ ]:
get_domain(trainSchema, 'race').value.append('Asian')

In [ ]:
assert 'Asian' in raceValues, 'False'

In [ ]:
testerAnomalies = validate_statistics(testerStats, trainSchema)

display_anomalies(testerAnomalies)